In [69]:
df = organize_data(config.train_data)

In [70]:
df = deal_data(df)

In [72]:
max(question_len)

145

In [63]:
model = Rnn(mode='GRU', input_size=2, hidden_size=3,dropout_p=0.2, bidirectional=True,layer_num=1, is_bn=False)

In [64]:
a = torch.randn(3, 2, 2)

In [65]:
mask = torch.Tensor([[1, 0, 0], [1,1,0]])

In [66]:
r, m = model(a, mask)

In [67]:
r

tensor([[[-0.1072,  0.0905,  0.1096,  0.0461,  0.0109, -0.0502],
         [ 0.0835, -0.1767, -0.0564, -0.2180, -0.2487,  0.1661]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.2516, -0.5362, -0.5890, -0.3020,  0.3067,  0.3731]]])

In [68]:
m

tensor([[ 1.,  0.],
        [ 1.,  1.]])

In [62]:
# coding = utf-8
# author = xy

import torch
from torch import nn


class Rnn(nn.Module):
    def __init__(self, mode, input_size, hidden_size, dropout_p, bidirectional, layer_num, is_bn):
        super(Rnn, self).__init__()

        self.mode = mode
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_p = dropout_p
        self.direction_num = bidirectional
        self.layer_num = layer_num
        self.is_bn = is_bn

        if mode == 'LSTM':
            self.rnn = nn.LSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=layer_num,
                bidirectional=bidirectional,
                dropout=dropout_p if layer_num > 1 else 0
            )
        elif mode == 'GRU':
            self.rnn = nn.GRU(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=layer_num,
                bidirectional=bidirectional,
                dropout=dropout_p if layer_num > 1 else 0
            )
        if is_bn:
            self.layer_norm = nn.LayerNorm(input_size)
        self.drop = nn.Dropout(p=dropout_p)
        self.reset_parameters()

    def reset_parameters(self):
        """ use xavier_uniform to initialize GRU/LSTM weights"""
        ih = (param for name, param in self.named_parameters() if 'weight_ih' in name)
        hh = (param for name, param in self.named_parameters() if 'weight_hh' in name)
        b = (param for name, param in self.named_parameters() if 'bias' in name)

        for t in ih:
            torch.nn.init.xavier_uniform_(t)
        for t in hh:
            torch.nn.init.orthogonal_(t)
        for t in b:
            torch.nn.init.constant_(t, 0)

    def forward(self, vec, mask):
        """
        :param vec: tensor (seq_len, batch_size, input_size)
        :param mask: tensor (batch_size, seq_len)
        :return: outputs: tensor (seq_len, batch_size, hidden_size*bidirectional)
        """
        # layer normalization
        if self.is_bn:
            seq_len, batch_size, input_size = vec.shape
            vec = vec.contiguous().view(-1, input_size)
            vec = self.layer_norm(vec)
            vec = vec.view(seq_len, batch_size, input_size)

        # dropout
        vec = self.drop(vec)

        # 这种方式
        lengths = mask.long().sum(1)
        lengths_sort, idx_sort = torch.sort(lengths, descending=True)
        _, idx_unsort = torch.sort(idx_sort)

        v_sort = vec.index_select(1, idx_sort)
        v_pack = nn.utils.rnn.pack_padded_sequence(v_sort, lengths_sort)
        outputs, _ = self.rnn(v_pack, None)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs.index_select(1, idx_unsort)

        # rnn, not state 注： 这种效果会比上面的好，但是不同batch_size, 输出结果不同
        # outputs, _ = self.rnn(vec, None)
        # outputs = mask.transpose(0, 1).unsqueeze(2) * outputs
        max_len = torch.max(lengths).item()
        seq_len = vec.size(0)
        if max_len != seq_len:
            pad_len = seq_len - max_len
            batch_size = vec.size(1)
            hidden_size = self.hidden_size * 2 if self.direction_num else self.hidden_size
            padding = outputs.new_zeros(pad_len, batch_size, hidden_size)
            outputs = torch.cat([outputs, padding], dim=0)
        
        
       
        
        
        
        return outputs
